"""

Constraints:
Each clothing item is given to exactly one child
Clothing size must match the chosen child’s size
Each child receives at least one summer item and one winter item


Objective:
◦ Let R = the sum of the retail prices for all of the clothing
◦ Let N = the number of children
◦ R/N = the fair share for each child
◦ Let Ci = the sum of the clothing prices for child i
◦ | R/N - Ci | = the amount over/under that child i received
◦ Your project code must compute a distribution that will provide the absolute minimum value for
D, where D is:


Signed Academic Integrity statement must be submitted


Your goal is to distribute them as fairly as possible, while ensuring that the clothes fit and that
each child gets at least one summer item and one winter item.
"""

In [328]:
fname = "example_input/ex1_2children_4clothes.txt"

#### 1. Load and Prepare data

In [329]:
with open(fname) as f:
    flines = f.readlines()
    clothes_records  = []
    children_records = []
    for fline in flines:
        if 'Clothes' in fline or 'Children' in fline:
            continue
        else:
            if fline[0] == 'A':
                cloth_record = fline.split("\t")
                clothes_records.append({
                                               'cloth_id': cloth_record[0],
                                               'cloth_size': cloth_record[1],
                                               'cloth_season': cloth_record[2],
                                               'cloth_price': cloth_record[3].strip()
                                       }
                                      )
            else:
                children_record = fline.split("\t")
                children_records.append(
                    {
                        'child_id': children_record[0],
                        'child_size': children_record[1].strip(),
                        'summer_clothes': [],
                        'winter_clothes': []
                    }
                )

In [330]:
import pandas as pd

In [331]:
pd.DataFrame(clothes_records)

,cloth_id,cloth_size,cloth_season,cloth_price
0,A1,XL,summer,31
1,A2,XL,winter,24
2,A3,L,summer,21
3,A4,ALL,winter,18


In [332]:
pd.DataFrame(children_records)

,child_id,child_size,summer_clothes,winter_clothes
0,C1,XL,[],[]
1,C2,L,[],[]


#### 2. Compute all static variables before starting the optimization algorithm

In [333]:
N = len(children_records) # the number of children
N

2

In [334]:
# R = the sum of the retail prices for all of the clothing
R = 0
for cloth_record in clothes_records:
    R += float(cloth_record['cloth_price'])
R

94.0

In [335]:
## R/N = the fair share for each child
r_by_n_ratio = R/N
r_by_n_ratio

47.0

#### 3. Matrix preparation

In [336]:
len(clothes_records), len(children_records)

(4, 2)

In [337]:
child_ids = [f"{each['child_id']}_{each['child_size']}" for each in children_records]
child_ids

['C1_XL', 'C2_L']

In [338]:
summer_cloth_ids = [f'SU_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='summer']
summer_cloth_ids

['SU_A1_XL_31', 'SU_A3_L_21']

In [339]:
winter_cloth_ids = [f'WI_{each["cloth_id"]}_{each["cloth_size"]}_{each["cloth_price"]}' for each in clothes_records if each['cloth_season']=='winter']
winter_cloth_ids

['WI_A2_XL_24', 'WI_A4_ALL_18']

In [340]:
summer_df = pd.DataFrame(index=child_ids, columns=summer_cloth_ids)
summer_df

,SU_A1_XL_31,SU_A3_L_21
C1_XL,NaN,NaN
C2_L,NaN,NaN


In [341]:
winter_df = pd.DataFrame(index=child_ids, columns=winter_cloth_ids)
winter_df

,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,NaN,NaN
C2_L,NaN,NaN


In [342]:
summer_df_i, summer_df_j = summer_df.shape
summer_df_i, summer_df_j

(2, 2)

In [343]:
summer_df_cols = summer_df.columns.tolist()
print(summer_df_cols)
summer_df_indices = summer_df.index.tolist()
print(summer_df_indices)

['SU_A1_XL_31', 'SU_A3_L_21']
['C1_XL', 'C2_L']


In [344]:
for i in range(summer_df_i):
    for j in range(summer_df_j):        
        cl_size = summer_df_cols[j].split("_")[2]
        cl_price = summer_df_cols[j].split("_")[3]
        cd_size = summer_df_indices[i].split("_")[1]
        if cl_size == cd_size or cl_size == "ALL":
            # assign this cloth to the child
            summer_df.loc[summer_df_indices[i], summer_df_cols[j]] = cl_price
                
            

In [345]:
summer_df

,SU_A1_XL_31,SU_A3_L_21
C1_XL,31,NaN
C2_L,NaN,21


In [346]:
winter_df_i, winter_df_j = winter_df.shape
winter_df_i, winter_df_j

(2, 2)

In [347]:
winter_df_cols = winter_df.columns.tolist()
print(winter_df_cols)
winter_df_indices = winter_df.index.tolist()
print(winter_df_indices)

['WI_A2_XL_24', 'WI_A4_ALL_18']
['C1_XL', 'C2_L']


In [348]:
for i in range(winter_df_i):
    for j in range(winter_df_j):
        cl_size = winter_df_cols[j].split("_")[2]
        cl_price = winter_df_cols[j].split("_")[3]
        cd_size = winter_df_indices[i].split("_")[1]
        if cl_size == cd_size or cl_size == "ALL":
            # assign this cloth to the child
            winter_df.loc[winter_df_indices[i], winter_df_cols[j]] = cl_price

In [349]:
summer_df

,SU_A1_XL_31,SU_A3_L_21
C1_XL,31,NaN
C2_L,NaN,21


In [350]:
winter_df

,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,24,18
C2_L,NaN,18


In [351]:
matrix_df = summer_df.join(winter_df)
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31,NaN,24,18
C2_L,NaN,21,NaN,18


#### 4. Algorithm

In [352]:
import numpy as np

In [353]:
matrix_df = matrix_df.apply(pd.to_numeric)
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31.0,NaN,24.0,18
C2_L,NaN,21.0,NaN,18


In [428]:
matrix_df.to_numpy()

array([[31., nan, 24., 18.],
       [nan, 21., nan, 18.]])

In [359]:
matrix_df_with_fair_cost = matrix_df - r_by_n_ratio
matrix_df_with_fair_cost # let us get back to this

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,-16.0,NaN,-23.0,-29.0
C2_L,NaN,-26.0,NaN,-29.0


In [317]:
matrix_df.fillna(-1, inplace=True)
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31.0,-1.0,24.0,18
C2_L,-1.0,21.0,-1.0,18


In [ ]:
C1 S = [31] W [24,18]
C2 S = [21] W [] -> not eligible

In [ ]:
C1 S = [31] W [24]
C2 S = [21] W [18] - > eligiblt

In [318]:
def create_and_get_path():
    path = {}
    for child_record in children_records:
        path[child_record['child_id']] = {'summer_clothes': [], 'winter_clothes': [], 'ci': 0,
                                         'summer_winter_constraint': False}
    return path

In [319]:
matrix_df_i, matrix_df_j = matrix_df.shape
matrix_df_i, matrix_df_j 

(2, 4)

In [320]:
matrix_df_indices = matrix_df.index.tolist()
matrix_df_cols = matrix_df.columns.tolist()

In [321]:
matrix_df_indices

['C1_XL', 'C2_L']

In [322]:
matrix_df_cols

['SU_A1_XL_31', 'SU_A3_L_21', 'WI_A2_XL_24', 'WI_A4_ALL_18']

In [388]:
matrix_df.loc[matrix_df_indices[1],matrix_df_cols[3]]

18

In [399]:
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31.0,NaN,24.0,18
C2_L,NaN,21.0,NaN,18


In [451]:
matrix_df = matrix_df.fillna(-1)

In [454]:
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31.0,-1.0,24.0,18
C2_L,-1.0,21.0,-1.0,18


In [619]:
def print_all_paths_helper(matrix_df, i, j, m, n, path, pi, pj, whitelisted_j):
    if pi == m:
        return 
    if pj == n - 1:
        # reached the right end of column
        # print and return
        if matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]] > 0:
            path[i][j] = matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]]
            # TODO: Replace this with whitelist
            for idx in range(matrix_df_i):
                if idx == i:
                    continue
                path[idx][j] = None
        local_minimum = 0
        for row_idx in range(matrix_df_i):
            ci_this_child = 0
            for col_idx in range(matrix_df_j):
                if path[row_idx][col_idx] is not None:
                    ci_this_child += path[row_idx][col_idx]
            local_minimum += abs(r_by_n_ratio - ci_this_child)
        print(local_minimum)
        for row in path:
            print(row)
                
        print("\n\n")
        # whitelist this j
        return
    # add them if only eligible
    # add current cell
    if matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]] > 0:
        path[i][j] = matrix_df.loc[matrix_df_indices[pi],matrix_df_cols[pj]]
        for idx in range(matrix_df_i):
            if idx == i:
                continue
            path[idx][j] = None
    else:
        if i + 1 < m:
            print_all_paths_helper(matrix_df, i+1, j , matrix_df_i, matrix_df_j, path, i + 1, j, whitelisted_j)
        if i - 1 >= 0:
            print_all_paths_helper(matrix_df, i-1, j, matrix_df_i, matrix_df_j, path, i - 1, j, whitelisted_j)


            
    # print all paths that are possible moving to next column
    #for ix in range(i, m):
    print_all_paths_helper(matrix_df, i, j + 1, matrix_df_i, matrix_df_j, path, pi, j + 1, whitelisted_j)
    #print_all_paths_helper(matrix_df, i+1, j + 1, matrix_df_i, matrix_df_j, path, i + 1, j + 1, whitelisted_j)
    

def print_all_paths(matrix_df, matrix_df_i, matrix_df_j):
    whitelisted_j = []
    path = [[None for j in range(matrix_df_j)] for i in range(matrix_df_i)]
    print_all_paths_helper(matrix_df, 0, 0, matrix_df_i, matrix_df_j, path, 0, 0, whitelisted_j)
        

In [620]:
print_all_paths(matrix_df, matrix_df_i, matrix_df_j)

52.0
[31.0, None, 24.0, 18]
[None, 21.0, None, None]



16.0
[31.0, None, 24.0, None]
[None, 21.0, None, 18]



52.0
[31.0, None, 24.0, 18]
[None, 21.0, None, None]





In [616]:
matrix_df

,SU_A1_XL_31,SU_A3_L_21,WI_A2_XL_24,WI_A4_ALL_18
C1_XL,31.0,-1.0,24.0,18
C2_L,-1.0,21.0,-1.0,18
